In [1]:
# load libraries
import pandas as pd
import os
import numpy as np
import datetime
from torch.utils.data import Dataset, DataLoader
from torch import save

In [2]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# change me, have to point to the Meteotrentino drive folder
wk =  "/content/drive/My Drive/10. Meteotrentino/"

# path to the datasets
dataset_path = wk+"DatasetPytorch/24+1_v2/"

# specify sensor type
sensor_type = "pioggia"

# specify name of the sensor type
sensor_dataset = "pioggia1"

# specify code of the weather station
sensor = "T0065"

file_name_test = sensor+"_test_"+sensor_dataset+"_tsf.pt"
file_name_train = sensor+"_test_"+sensor_dataset+"_tsf.pt"

# extrapolate the paths
file_test_to_read_path = os.path.join(wk, dataset_path, sensor_type, file_name_test)
file_train_to_read_path = os.path.join(wk, dataset_path, sensor_type, file_name_train)

In [5]:
import torch
import sys
# clone the libraries if not done 
!git clone https://github.com/eliazonta/AI-Challenge-HIT
# append to sys.path to load other modules, needed for the custom dataset
sys.path.append("/content/AI-Challenge-HIT/python_code/notebooks/PytorchFormatter/")

from custom_datasets_pytorch import CustomDataset

In [6]:
from torch import save,load
dataset = {}
splits = ['train','test']
shuffle = {'train':True,'test':False}
# here load a pre-existing dataset
dataset['train'] = load(file_train_to_read_path)
dataset['test'] = load(file_test_to_read_path)
# mean and std of train (target is small)
mean = dataset['train'].features.mean()
std = dataset['train'].features.std()
# for training, standardize the dataset
dataset['train'].features = (dataset['train'].features - mean)/std
dataset['train'].target = (dataset['train'].target - mean)/std
dataset['test'].features = (dataset['test'].features - mean)/std
dataset['test'].target = (dataset['test'].target - mean)/std

# definition of the dataloader
b_size = {'train':64,'test':len(dataset['test'])}
dataloader = {x: torch.utils.data.DataLoader(dataset=dataset[x],
                                            batch_size=b_size[x],
                                            shuffle=shuffle[x],
                                            collate_fn=lambda x: x,
                                            drop_last=True)
            for x in splits}

In [8]:
# split required if dataset is too fat
SPLIT_NUMBER = 50

In [2]:
from collections.abc import Sequence
from importlib import reload  # Python 3.4+
import torch
import sys
from torch import nn
# clone the libraries if not done 
#!git clone https://github.com/eliazonta/AI-Challenge-HIT
sys.path.append("/content/AI-Challenge-HIT/python_code/notebooks/AnomalyDetection/")
import networks
LSTMs = reload(networks)
from networks import LSTM
from tqdm import tqdm

# specify the device ("cuda" for GPU, "cpu" for CPU)
device = torch.device("cuda")

# get dome parameters from the dataset
sequence_length = dataset['train'][0]['features'].size(0)
pred_length = dataset['train'][0]['target'].size(0)
# Instantiate the model with hyperparameters
model = LSTM(input_size=1, output_size=1, hidden_dim=32, num_layers=3, device = device)
train_loss = []
test_loss = []

# We'll also set the model to the device that we defined earlier
model = model.to(device)

# Define hyperparameters
lr = 1e-5

# Define Loss and Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

n_epochs = 30
for epoch in range(1, n_epochs + 1):
    model.train()
    loss_train_batch = []
    for data in tqdm(dataloader['train'], leave=True, total=len(dataloader['train'])):
      # preparing the input
      batch_input_features = data.features.to(device)
      target_features = data.target.to(device)
      # feed the model and make the prediction
      output = model(batch_input_features,pred_length)
      output = output.to(device)[:,-pred_length:]
      # compute the loss
      loss_train = criterion(output, target_features)
      # backpropagation and gradients
      loss_train.backward() 
      # Update of the weights accordingly
      optimizer.step() 
      # Append the loss to batch list
      loss_train_batch.append(loss_train.item())
    
    # average on bath train loss
    mean_train_loss = np.mean(loss_train_batch)

    # test-time
    model.eval()
    data_test = next(iter(dataloader['test']))
    test_input_features = data_test.features.to(device)
    test_target_features = data_test.target.to(device)
    
    # split when it is too fat (out of memory)
    test_input_features_splitted = np.array_split(test_input_features, SPLIT_NUMBER)
    output_test_list = []
    for input_feature_split_subset in test_input_features_splitted:
      output_test_list.append(model(input_feature_split_subset,pred_length)[:,-pred_length:])
    output_test_flat = [item for item in output_test_list]
    output_test = torch.cat(output_test_flat)
    # compute loss on test set for validation on unseen data
    loss_test = criterion(output_test, test_target_features)
    
    train_loss.append(mean_train_loss)
    test_loss.append(loss_test.item())
    print("Epoch = " + str(epoch))
    print("\tTraining Loss: {:.4f}".format(mean_train_loss))
    print("\tTest Loss: {:.4f}".format(loss_test.item()))

    # stop criterion on the train loss value
    if mean_train_loss < 0.010:
        break

# plot the training performance (todo: use W&B)
import matplotlib.pyplot as plt
plt.plot(train_loss,'o-',label="train")
plt.plot(test_loss,'o-',label="test")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

NameError: name 'dataset' is not defined

In [12]:
# save model once trained
torch.save(model.state_dict(),
           os.path.join(wk, "TrainedModels/", sensor+"_"+sensor_type+"_model.pt")
)